<a href="https://colab.research.google.com/github/OlehVakulchyk/DataScience/blob/main/Hw3_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import pandas as pd
import numpy as np

file_id = '1-rAa4XT4_fI0dOBlMNuE6a7jB0wln_Qo'
url = f'https://drive.google.com/uc?id={file_id}'
price_house = pd.read_csv(url)
price_house

,price,area,bedrooms,bathrooms,stories,mainroad,guestroom,basement,hotwaterheating,airconditioning,parking,prefarea,furnishingstatus
0,13300000,7420,4,2,3,yes,no,no,no,yes,2,yes,furnished
1,12250000,8960,4,4,4,yes,no,no,no,yes,3,no,furnished
2,12250000,9960,3,2,2,yes,no,yes,no,no,2,yes,semi-furnished
3,12215000,7500,4,2,2,yes,no,yes,no,yes,3,yes,furnished
4,11410000,7420,4,1,2,yes,yes,yes,no,yes,2,no,furnished
...,...,...,...,...,...,...,...,...,...,...,...,...,...
540,1820000,3000,2,1,1,yes,no,yes,no,no,2,no,unfurnished
541,1767150,2400,3,1,1,no,no,no,no,no,0,no,semi-furnished
542,1750000,3620,2,1,1,yes,no,no,no,no,0,no,unfurnished
543,1750000,2910,3,1,1,no,no,no,no,no,0,no,furnished


напишіть функцію гіпотези лінійної регресії у векторному вигляді;

In [3]:
# X -  матриця ознак з першим стовпчиком з одиниць
# w - вектор ваг
def h(X, w):
  return X @ w

створіть функцію для обчислення функції втрат у векторному вигляді;

In [4]:
def mse(X, y, w):
  return (((h(X, w))-y) @ (h(X, w))-y) / (2 * len(X))

реалізуйте один крок градієнтного спуску;

In [5]:
def gradient_step(X, y, w, learning_rate):
  gradient_mse = (X.T @ (h(X, w) - y))/len(X)
  w = w - learning_rate * gradient_mse
  return w

знайдіть найкращі параметри $\vec{w}$ для датасету прогнозуючу ціну на будинок залежно від площі, кількості ванних кімнат та кількості спалень;

нормалізація даних

In [6]:
from sklearn.preprocessing import MinMaxScaler, StandardScaler
Xx = price_house[['price', 'area', 'bedrooms', 'bathrooms']]
Xx_minmax = pd.DataFrame(MinMaxScaler().fit_transform(Xx))
display(Xx_minmax)
Xx_standard = pd.DataFrame(StandardScaler().fit_transform(Xx))
display(Xx_standard)

,0,1,2,3
0,1.000000,0.396564,0.6,0.333333
1,0.909091,0.502405,0.6,1.000000
2,0.909091,0.571134,0.4,0.333333
3,0.906061,0.402062,0.6,0.333333
4,0.836364,0.396564,0.6,0.000000
...,...,...,...,...
540,0.006061,0.092784,0.2,0.000000
541,0.001485,0.051546,0.4,0.000000
542,0.000000,0.135395,0.2,0.000000
543,0.000000,0.086598,0.4,0.000000


,0,1,2,3
0,4.566365,1.046726,1.403419,1.421812
1,4.004484,1.757010,1.403419,5.405809
2,4.004484,2.218232,0.047278,1.421812
3,3.985755,1.083624,1.403419,1.421812
4,3.554979,1.046726,1.403419,-0.570187
...,...,...,...,...
540,-1.576868,-0.991879,-1.308863,-0.570187
541,-1.605149,-1.268613,0.047278,-0.570187
542,-1.614327,-0.705921,-1.308863,-0.570187
543,-1.614327,-1.033389,0.047278,-0.570187


In [15]:
y = Xx_minmax[0]
X = Xx_minmax[[1, 2, 3]]

x_1 = pd.Series(np.ones((len(X),), dtype=int))
X_1 = pd.concat([x_1,X], axis=1)

w = np.array([0, 1, 1, 1])
learning_rate = 1e-1
max_iteration = 5000
precision = 1e-7
previous_mse = 0

for i in range(max_iteration):
  if sum(abs(mse(X_1, y, w) - previous_mse)) <= precision:
    print(i)
    break
  previous_mse = mse(X_1, y, w)
  w = gradient_step(X_1, y, w, learning_rate)
w


4062


0    0.042449
1    0.477324
2    0.177119
3    0.359322
dtype: float64

In [8]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression().fit(X, y)
regressor.coef_, regressor.intercept_

(array([0.47714269, 0.17611257, 0.36001286]), 0.04282739976995403)

знайдіть ці ж параметри за допомогою аналітичного рішення;

In [14]:
# pandas
x_1 = pd.Series(np.ones((len(X),), dtype=int))
X_1 = pd.concat([x_1,X], axis=1)
w = np.linalg.inv(X_1.T@X_1)@X_1.T@y
w

0    0.042827
1    0.477143
2    0.176113
3    0.360013
dtype: float64

In [10]:
# numpy
X1 = np.array(X_1)
w = np.linalg.inv(X1.T@X1)@X1.T@y
w

array([0.0428274 , 0.47714269, 0.17611257, 0.36001286])

порівняйте отримані результати.

Аналітичне рішення та LinearRegression дають однакові результати.
Щоб отримати таку ж точність для "своєї" програми треба підбирати вхідні параметри(w) та learning_rate, max_iteration, precision